In [14]:
import pandas as pd
from pygbif import registry
from requests_cache import remove_expired_responses
import re

In [15]:
def get_description(uuid):
    try:
        s = registry.dataset_search(uuid)['results'][0]['description']
    except:
        print(f'Cannot get description for uuid {uuid}')
        return f'Cannot get description for uuid {uuid}'
    print(s)
    print()
    s = s.replace('This dataset contains the digitized treatments in Plazi based on the original book chapter ', '')
    doi = extract_doi(s)
    doi_link = f'<a href="{doi}">{doi}</a>'
    s = f'{s}   {doi_link}'    
    return s

#get_description(df.uuid[10])

In [16]:
def extract_doi(text):
    doi_re = re.compile("10.(\d)+/([^(\s\>\"\<)])+")
    m = doi_re.search(text)
    return m.group(0)
    
#extract_doi(get_description(df.uuid[1]))

In [17]:
def get_gbif_link(uuid):
    s = f'https://www.gbif.org/dataset/search?q={uuid}'
    link = f'<a href="{s}">{s}</a>'
    return link

#get_gbif_link(df.uuid[1])

In [18]:
df = pd.read_csv('data/guam_uuids.txt', delimiter='\t', header=None) 
df.drop([0,3], axis='columns', inplace=True)
df.columns = ['uuid', 'imf']
df['gbif'] = df.apply(lambda x: get_gbif_link(x.uuid), axis=1)
df['zenodo'] = df.apply(lambda x: get_description(x.uuid), axis=1)
df.to_html('data/uuid_list.html', escape=False)

This dataset contains the digitized treatments in Plazi based on the original book chapter Zimmerman, Elwood C. (1942): Curculionidae of Guam. In: Insects of Guam I. Honolulu, Hawaii: Bernice P. Bishop Museum: 73-146, DOI: http://doi.org/10.5281/zenodo.5159964

This dataset contains the digitized treatments in Plazi based on the original book chapter Swezey, O. H. (1942): Culicidae of Guam. In: Insects of Guam I. Honolulu, Hawaii: Bernice P. Bishop Museum: 199-200, DOI: http://doi.org/10.5281/zenodo.5173998

This dataset contains the digitized treatments in Plazi based on the original book chapter SchedL, Karl E. (1942): Barkbeetles of Guam. In: Insects of Guam I. Honolulu, Hawaii: Bernice P. Bishop Museum: 147-149, DOI: http://doi.org/10.5281/zenodo.5160072

This dataset contains the digitized treatments in Plazi based on the original book chapter Banks, Nathan (1942): Neuropteroid Insects from Guam. In: Insects of Guam I. Honolulu, Hawaii: Bernice P. Bishop Museum: 25-30, DOI: http:/

In [19]:
df

,uuid,imf,gbif,zenodo
0,2A01FF71FFF051065F5BAF5AFFFAFF96,InsectsOfGuamI.172.73-146.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Zimmerman, Elwood C. (1942): Curculionidae of ..."
1,4125F147872BFF9DFFF5FFAC140DFF83,InsectsOfGuamI.172.199-200.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Swezey, O. H. (1942): Culicidae of Guam. In: I..."
2,8A034648715AFFEADD11FF809F446C3F,InsectsOfGuamI.172.147-149.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","SchedL, Karl E. (1942): Barkbeetles of Guam. I..."
3,9668142AFFAC355BFFD8FFF0FFC6FFA4,InsectsOfGuamI.172.25-30.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Banks, Nathan (1942): Neuropteroid Insects fro..."
4,A676FD1EF22D3F34FF8F8907FFDAFC58,InsectsOfGuamI.172.191-194.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Cockerell, T. D. A. (1942): Halictine Bees fro..."
5,AD79FFBAEA10FFDBFFFE8726FFBFFFFE,InsectsOfGuamI.172.65-72.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Zimmerman, Elwood C. (1942): Anthribidae Of Gu..."
6,B525F8594A7852476D53FFF00A6FFFD2,InsectsOfGuamI.172.184-187.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Swezey, O. H. (1942): Wasps of Guam. In: Insec..."
7,BE7D4354FFEEFF8BFFBDFFBC0B359010,InsectsOfGuamI.172.53-55.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Van Zwaluwenburg, R. H. (1942): Elaterid And E..."
8,C5751610FFAD3E7CE078FFB1FFCFFF82,InsectsOfGuamI.172.39-40.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Swezry, O. H. (1942): Sphingidae Of Guam. In: ..."
9,CB52FF9DFFBE1B09FFDAFFB507064636,InsectsOfGuamI.172.150-171.pdf.imf,"<a href=""https://www.gbif.org/dataset/search?q...","Swezey, O. H. (1942): Miscellaneous Families o..."
